## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-18-22-51-08 +0000,nyt,Congress Overwhelmingly Approves Releasing Eps...,https://www.nytimes.com/2025/11/18/us/politics...
1,2025-11-18-22-48-39 +0000,nypost,D4vd being eyed as suspect in death of teen wh...,https://nypost.com/2025/11/18/us-news/d4vd-bei...
2,2025-11-18-22-47-13 +0000,nypost,Goldman Sachs employees shocked after exec cau...,https://nypost.com/2025/11/18/business/goldman...
3,2025-11-18-22-44-48 +0000,nypost,Meet the glamorous economist Larry Summers ask...,https://nypost.com/2025/11/18/us-news/meet-eco...
4,2025-11-18-22-41-55 +0000,nypost,LI couple admits to hacking up roommates with ...,https://nypost.com/2025/11/18/us-news/li-coupl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2331/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
53,trump,50
4,epstein,24
132,house,16
55,new,16
82,saudi,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
33,2025-11-18-21-57-00 +0000,wsj,The House voted to force a public release of f...,https://www.wsj.com/politics/policy/epstein-fi...,153
307,2025-11-18-02-00-00 +0000,wsj,President Trump’s grip on the GOP is showing s...,https://www.wsj.com/politics/policy/trumps-gri...,122
22,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...,122
209,2025-11-18-13-24-46 +0000,cbc,Saudi crown prince MBS unaware of Khashoggi ki...,https://www.cbc.ca/news/world/washington-saudi...,121
10,2025-11-18-22-36-00 +0000,wsj,A federal court has blocked the use of a new T...,https://www.wsj.com/politics/policy/texas-new-...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
33,153,2025-11-18-21-57-00 +0000,wsj,The House voted to force a public release of f...,https://www.wsj.com/politics/policy/epstein-fi...
104,55,2025-11-18-19-20-12 +0000,nyt,"For the Saudi crown prince, international crit...",https://www.nytimes.com/live/2025/11/18/world/...
22,48,2025-11-18-22-15-00 +0000,wsj,"Once one of Trump’s most strident MAGA allies,...",https://www.wsj.com/politics/policy/how-marjor...
307,41,2025-11-18-02-00-00 +0000,wsj,President Trump’s grip on the GOP is showing s...,https://www.wsj.com/politics/policy/trumps-gri...
124,39,2025-11-18-18-04-02 +0000,nyt,Zelensky Will Try to Revive Peace Talks With R...,https://www.nytimes.com/2025/11/18/world/europ...
113,37,2025-11-18-18-52-13 +0000,nypost,Google’s Sundar Pichai warns of AI spending ‘i...,https://nypost.com/2025/11/18/business/google-...
1,36,2025-11-18-22-48-39 +0000,nypost,D4vd being eyed as suspect in death of teen wh...,https://nypost.com/2025/11/18/us-news/d4vd-bei...
41,36,2025-11-18-21-46-47 +0000,latimes,"'Played with fire, got burned': GOP control of...",https://www.latimes.com/politics/story/2025-11...
232,33,2025-11-18-11-37-32 +0000,nypost,James Comey trial judge stays order for feds t...,https://nypost.com/2025/11/18/us-news/james-co...
83,33,2025-11-18-20-14-16 +0000,nypost,House votes to denounce Rep. Chuy Garcia for ‘...,https://nypost.com/2025/11/18/us-news/house-vo...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
